In [2]:
!pip install transformers datasets evaluate -q --use-deprecated=legacy-resolver

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate

In [4]:
dataset = load_dataset("imdb")
# Use a small subset of data for faster execution
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
small_test_dataset = dataset["test"].shuffle(seed=42).select(range(500))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
#Load the pre-trained tokenizer and model
pretrained_model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

encoded_train_dataset = small_train_dataset.map(preprocess_function, batched=True)
encoded_test_dataset = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [11]:
pretrained_model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=2)
trainer = Trainer(
    model=pretrained_model,
    tokenizer=tokenizer,
    eval_dataset=encoded_test_dataset,
    compute_metrics=compute_metrics # Pass the compute_metrics function to the Trainer
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-11-63d209418c7d>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
# Evaluation function
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
print("Evaluating pre-trained model...")
pretrained_results = trainer.evaluate()
print(f"Accuracy (Pre-trained): {pretrained_results['eval_accuracy']}")

Evaluating pre-trained model...


Accuracy (Pre-trained): 0.474


In [14]:
#Fine-tune the model
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',
    save_strategy="no",
    report_to="none"  # Disable W&B or other reporting integrations
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=2)
trainer = Trainer(
    model=fine_tuned_model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-15-e25c18e946ca>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
print("Fine-tuning the model...")
trainer.train()

Fine-tuning the model...


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.337159,0.852000
2,No log,0.346026,0.864000


TrainOutput(global_step=250, training_loss=0.3720553894042969, metrics={'train_runtime': 112.8806, 'train_samples_per_second': 17.718, 'train_steps_per_second': 2.215, 'total_flos': 264934797312000.0, 'train_loss': 0.3720553894042969, 'epoch': 2.0})

In [17]:
#Evaluate the fine-tuned model
print("Evaluating fine-tuned model...")
fine_tuned_results = trainer.evaluate()
print(f"Accuracy (Fine-tuned): {fine_tuned_results['eval_accuracy']}")

Evaluating fine-tuned model...


Accuracy (Fine-tuned): 0.864


In [26]:
#Mock example comparison
def predict_sentiment(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(axis=-1).item()
    return "Positive" if predictions == 1 else "Negative"

mock_review_1 = "The movie was absolutely fantastic! I loved every moment of it."
print("\nMock Example:")
print(f"Review: {mock_review_1}")


Mock Example:
Review: The movie was absolutely fantastic! I loved every moment of it.


In [27]:
# Predict with pre-trained model
pretrained_sentiment = predict_sentiment(pretrained_model, tokenizer, mock_review_1)
print(f"Prediction (Pre-trained): {pretrained_sentiment}")


Prediction (Pre-trained): Positive


In [28]:
fine_tuned_sentiment = predict_sentiment(fine_tuned_model, tokenizer, mock_review_1)
print(f"Prediction (Fine-tuned): {fine_tuned_sentiment}")


Prediction (Fine-tuned): Positive


In [29]:
mock_review_2 = "The movie was absolutely fantastic?! I hated every moment of it."
print("\nMock Example:")
print(f"Review: {mock_review_1}")


Mock Example:
Review: The movie was absolutely fantastic! I loved every moment of it.


In [30]:
# Predict with pre-trained model
pretrained_sentiment = predict_sentiment(pretrained_model, tokenizer, mock_review_2)
print(f"Prediction (Pre-trained): {pretrained_sentiment}")

Prediction (Pre-trained): Positive


In [31]:
fine_tuned_sentiment = predict_sentiment(fine_tuned_model, tokenizer, mock_review_2)
print(f"Prediction (Fine-tuned): {fine_tuned_sentiment}")


Prediction (Fine-tuned): Negative


In [25]:
print("\nComparison:")
print(f"Accuracy (Pre-trained): {pretrained_results['eval_accuracy']}")
print(f"Accuracy (Fine-tuned): {fine_tuned_results['eval_accuracy']}")


Comparison:
Accuracy (Pre-trained): 0.474
Accuracy (Fine-tuned): 0.864
